In [1]:
import os
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

PROJECT_ID   = os.environ.get("PROJECT_ID")
SRC_DATASET  = os.environ.get("SILVER_DATASET")
DST_DATASET  = os.environ.get("GOLD_DATASET")
LOCATION     = os.environ.get("BQ_LOCATION")
STRIP_PREFIX = os.environ.get("STRIP_PREFIX")  # remove nk_*

bq = bigquery.Client(project=PROJECT_ID)

print("PROJECT_ID:", PROJECT_ID)
print("SRC_DATASET:", SRC_DATASET)
print("DST_DATASET:", DST_DATASET)
print("LOCATION:", LOCATION)
print("STRIP_PREFIX:", STRIP_PREFIX)


c:\Users\victo\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


PROJECT_ID: rq-pharma-data-lab-26k9
SRC_DATASET: silver_bps
DST_DATASET: bps
LOCATION: US
STRIP_PREFIX: nk_


In [2]:
def ensure_dataset(project: str, dataset: str, location: str):
    ds_id = f"{project}.{dataset}"
    try:
        bq.get_dataset(ds_id)
        return
    except NotFound:
        ds = bigquery.Dataset(ds_id)
        ds.location = location
        bq.create_dataset(ds)
        print(f"✅ Dataset criado: {ds_id} (location={location})")

In [3]:
def dataset_location(project: str, dataset: str, fallback: str) -> str:
    try:
        ds = bq.get_dataset(f"{project}.{dataset}")
        return ds.location or fallback
    except NotFound:
        return fallback

def q(col: str) -> str:
    return f"`{col}`"

def run_sql(sql: str):
    job = bq.query(sql, location=LOCATION)
    return job.result()

In [4]:
def build_sql_for_table(table_obj: bigquery.Table) -> str:
    src = f"{PROJECT_ID}.{SRC_DATASET}.{table_obj.table_id}"
    dst = f"{PROJECT_ID}.{DST_DATASET}.{table_obj.table_id}"

    # colunas top-level
    cols = [f.name for f in table_obj.schema]
    keep = [c for c in cols if not c.startswith(STRIP_PREFIX)]
    if not keep:
        raise RuntimeError(f"{src} ficaria sem colunas após remover {STRIP_PREFIX}*")

    select_list = ",\n  ".join(q(c) for c in keep)

    # VIEW / MATERIALIZED_VIEW -> cria VIEW normal no destino
    if (table_obj.table_type or "").upper() in ("VIEW", "MATERIALIZED_VIEW"):
        return f"""
CREATE OR REPLACE VIEW `{dst}` AS
SELECT
  {select_list}
FROM `{src}`;
""".strip()

    # TABLE -> tenta preservar partition/cluster (se for por coluna)
    partition_sql = ""
    if table_obj.time_partitioning and table_obj.time_partitioning.field:
        partition_sql = f"\nPARTITION BY {q(table_obj.time_partitioning.field)}"

    if table_obj.range_partitioning:
        rp = table_obj.range_partitioning
        partition_sql = (
            f"\nPARTITION BY RANGE_BUCKET({q(rp.field)}, "
            f"GENERATE_ARRAY({rp.range_.start}, {rp.range_.end}, {rp.range_.interval}))"
        )

    cluster_sql = ""
    if table_obj.clustering_fields:
        cluster_sql = "\nCLUSTER BY " + ", ".join(q(c) for c in table_obj.clustering_fields)

    return f"""
CREATE OR REPLACE TABLE `{dst}`{partition_sql}{cluster_sql} AS
SELECT
  {select_list}
FROM `{src}`;
""".strip()

In [5]:
src_loc = dataset_location(PROJECT_ID, SRC_DATASET, LOCATION)
LOCATION = src_loc  # garante mesma location do dataset source

ensure_dataset(PROJECT_ID, DST_DATASET, LOCATION)

tables = list(bq.list_tables(f"{PROJECT_ID}.{SRC_DATASET}"))
print(f"📦 Tabelas encontradas em {PROJECT_ID}.{SRC_DATASET}: {len(tables)}")

ok = 0
fail = 0

for t in tables:
    try:
        tbl = bq.get_table(t.reference)
        sql = build_sql_for_table(tbl)
        print(f"\n--- {tbl.table_type} {tbl.table_id} ---")
        run_sql(sql)
        print(f"✅ OK: {PROJECT_ID}.{DST_DATASET}.{tbl.table_id}")
        ok += 1
    except Exception as e:
        print(f"❌ FAIL {t.table_id}: {e}")
        fail += 1

print("\nResumo -> OK:", ok, "| FAIL:", fail)

📦 Tabelas encontradas em rq-pharma-data-lab-26k9.silver_bps: 9

--- TABLE dm_fabricante ---
✅ OK: rq-pharma-data-lab-26k9.bps.dm_fabricante

--- TABLE dm_fornecedor ---
✅ OK: rq-pharma-data-lab-26k9.bps.dm_fornecedor

--- TABLE dm_instituicao ---
✅ OK: rq-pharma-data-lab-26k9.bps.dm_instituicao

--- TABLE dm_modalidade_compra ---
✅ OK: rq-pharma-data-lab-26k9.bps.dm_modalidade_compra

--- TABLE dm_municipio ---
✅ OK: rq-pharma-data-lab-26k9.bps.dm_municipio

--- TABLE dm_produto ---
✅ OK: rq-pharma-data-lab-26k9.bps.dm_produto

--- TABLE dm_tipo_compra ---
✅ OK: rq-pharma-data-lab-26k9.bps.dm_tipo_compra

--- TABLE dm_uf ---
✅ OK: rq-pharma-data-lab-26k9.bps.dm_uf

--- TABLE fato_bps ---
✅ OK: rq-pharma-data-lab-26k9.bps.fato_bps

Resumo -> OK: 9 | FAIL: 0
